In [5]:
# STEP 1 : Data Preparation
import pandas as pd
from sklearn.model_selection import train_test_split

data = pd.read_csv("D:\IMPORTANT\Programming\AIML\Data\melbourne_data.csv")
# print(data.iloc[:5, :3])
# print(data.columns)
print(len(data))

y = data.Price
iFeatures = [
        'Suburb', 'Address', 'Rooms', 'Type','SellerG',
       'Date', 'Distance', 'Postcode', 'Bedroom2', 'Bathroom', 'Car',
       'Landsize', 'BuildingArea', 'YearBuilt', 'CouncilArea', 'Lattitude',
       'Longtitude', 'Regionname', 'Propertycount']

X = data[iFeatures]

trainX, valX, trainy, valy = train_test_split(X, y, random_state = 1)

13580


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error